In [1]:
import re

INPUT_FILE = 'input_d8.txt'

class Compiler:
    '''
    Basic compiler implementation for AoC Day8
    '''
    
    def __init__(self, input_file):
        self.program = []
        self.accumulator = 0
        self.next_line = 0
        self.load_file(input_file)        
    
    def load_file(self, input_file):
        '''
        Loads the program contained in the specified file
        :param input_file:
        '''
        with open(input_file) as f:
            for line in f.readlines():
                try:
                    self.program.append(Compiler.read_instruction(line))
                except ValueError as ve:
                    print(f'Error on line {len(self.program)}: {line}')
    
    def reset(self):
        '''
        Resets the internal registers to 0
        '''
        self.accumulator = 0
        self.next_line = 0
                
    def print_line(self, line_num, print_registers=False):
        '''
        Prints the specified line
        :param line_num:
        :param print_registers: when True, prints the current accumulator and next_line values
        '''
        instruct, param = self.program[line_num]
        s = f'{line_num}] {instruct} {param}'
        if print_registers:
            s += f' [acc={self.accumulator} next={self.next_line}]'
        print(s)
    
        
    def print_program(self):
        '''
        Prints the entire loaded program with line numbers
        '''
        for line in range(len(self.program)):
            self.print_line(line)
            
    def exec_line(self, line_num):
        '''
        Executes the instruction at the specified line and updates internal
        state accordingly
        :param line_num:
        '''
        instruction, param = self.program[line_num]
        
        if instruction == 'jmp':
            # Jump instructions based on the param
            self.next_line += param
        elif instruction == 'acc':
            # Update the accumulator by param, advance 1 line
            self.accumulator += param
            self.next_line += 1
        elif instruction == 'nop':
            # Ignore param, advance 1 line
            self.next_line += 1
        else:
            raise ValueError("Invalid instruction")
            
        
    def run_debug(self, verbose=False):
        '''
        Part 1 solution - runs the program until any instruction is executed twice
        :param verbose: Prints each line as executed
        :return: The value of the accumulator before any instruction is executed again
        '''
        self.reset()
        lines_executed = set()
        while True:
            if self.next_line > len(self.program):
                # Program complete, return accumulator
                return self.accumulator            
            elif self.next_line in lines_executed:
                # "break" when some line is about to be executed twice, return the accumulator value
                print(f'BREAK: {self.next_line} already ran!')
                return self.accumulator
            else:
                this_line = self.next_line
                lines_executed.add(this_line)
                self.exec_line(this_line)
                if verbose:
                    self.print_line(this_line, True)
                    
    def run_to_end(self):
        '''
        Part 2 solution -- recursive search to find the execution path
        that leads to the end of the program. If the path as written doesn't
        reach the end of the program, 
        :return: the value of the accumulator at the end, or None if end not reached
        '''
        self.visited = set()
        return self.rec_run_to_end(0, 0)
    
    def rec_run_to_end(self, line, accumulator):
        '''
        Recursive search for run_to_end()
        '''
        if line == len(self.program):
            # If we have reached the line after the last line, success!
            print(f'Reached end of program')
            return accumulator
        elif line in self.visited:
            # If line has already been visited or doesn't exist, this is a loop/dead end
            print(f'{line} already visited')
            return False
        elif line > len(self.program) or line < 0:
            print(f'{line} out of bounds')
            return False
        else:
            print(f'exec line {line}')
            # Otherwise, load this line and search its execution paths
            self.visited.add(line)
            instruction, param = self.program[line]
        
        if instruction == 'acc':
            next_acc = accumulator + param
            next_line = line + 1
            alt_line = None           # no alternative subtree to search for acc
        elif instruction == 'nop':
            next_line = line + 1
            alt_line = line + param   # the next line if we swap nop/jmp
            next_acc = accumulator
        elif instruction == 'jmp':
            next_line = line + param
            alt_line = line + 1       # the next line if we swap jmp/nop
            next_acc = accumulator
        else:
            raise ValueError("Invalid instruction!")
        
        # "run" the next line based on the instruction
        result = self.rec_run_to_end(next_line, next_acc)
        
        if not result and alt_line:
            # if the instruction didn't yield a path to the end, search the alternate path when one exists
            print(f'Try swapping line {line}, acc={next_acc}')
            self.alt_visited = set()
            return self.rec_alt_run(alt_line, next_acc)
        else:
            return result
        
    def rec_alt_run(self, line, accumulator):
        '''
        Searches an alternate execution path, with no instruction swaps, 
        since the swap already happened above the point in the chain where
        this function was called (see above)
        :param line:
        :param accumulator:
        '''
        if line == len(self.program):
            # If we have reached the line after the last line, success!
            print(f'Reached end of program')
            return accumulator
        elif line in self.alt_visited:
            # If line has already been visited or doesn't exist, this is a loop/dead end
            print(f'{line} already visited')
            return False
        elif line > len(self.program) or line < 0:
            print(f'{line} out of bounds')
            return False
        else:
            print(f'exec line {line}')
            # Otherwise, load this line and search its execution paths
            self.alt_visited.add(line)
            instruction, param = self.program[line]
        
        if instruction == 'acc':
            return self.rec_alt_run(line+1, accumulator + param)
        elif instruction == 'jmp':
            return self.rec_alt_run(line+param, accumulator)
        else:
            return self.rec_alt_run(line+1, accumulator)
        

    
    @staticmethod
    def read_instruction(line):
        '''
        Reads a string instruction and extracts the operation and argument
        :param instruction: string containing instruction and arg
        :return: Tuple of (instruction, arg)
        '''
        match = re.match(r"(acc|jmp|nop) ([+-]\d+)", line)
        if match:
            return (match[1], int(match[2]))
        else:
            raise ValueError("Invalid instruction")


In [2]:
# Day 8 example
ex = Compiler('input2_d8.txt')
ex.print_program()

0] nop 0
1] acc 1
2] jmp 4
3] acc 3
4] jmp -3
5] acc -99
6] acc 1
7] jmp -4
8] acc 6


In [3]:
# Test the example
assert ex.run_debug(True) == 5

0] nop 0 [acc=0 next=1]
1] acc 1 [acc=1 next=2]
2] jmp 4 [acc=1 next=6]
6] acc 1 [acc=2 next=7]
7] jmp -4 [acc=2 next=3]
3] acc 3 [acc=5 next=4]
4] jmp -3 [acc=5 next=1]
BREAK: 1 already ran!


In [4]:
# Part 1 solution
p1 = Compiler(INPUT_FILE)
p1.run_debug()

BREAK: 296 already ran!


1594

### Part 2
After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6

If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6

After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [5]:
ex.run_to_end()

exec line 0
exec line 1
exec line 2
exec line 6
exec line 7
exec line 3
exec line 4
1 already visited
Try swapping line 4, acc=5
exec line 5
exec line 6
exec line 7
exec line 3
exec line 4
exec line 1
exec line 2
6 already visited
Try swapping line 7, acc=2
exec line 8
Reached end of program


8

In [6]:
# Part 2 solution
p1.run_to_end()

exec line 0
exec line 1
exec line 2
exec line 3
exec line 4
exec line 195
exec line 196
exec line 197
exec line 198
exec line 199
exec line 449
exec line 296
exec line 297
exec line 298
exec line 112
exec line 113
exec line 114
exec line 115
exec line 52
exec line 53
exec line 54
exec line 104
exec line 105
exec line 106
exec line 107
exec line 568
exec line 569
exec line 570
exec line 130
exec line 131
exec line 132
exec line 557
exec line 558
exec line 559
exec line 560
exec line 384
exec line 385
exec line 174
exec line 175
exec line 176
exec line 177
exec line 84
exec line 85
exec line 600
exec line 601
exec line 602
exec line 537
exec line 538
exec line 43
exec line 44
exec line 45
exec line 46
exec line 331
exec line 332
exec line 333
exec line 159
exec line 160
exec line 161
exec line 162
exec line 163
exec line 76
exec line 77
exec line 78
exec line 79
exec line 80
exec line 126
exec line 127
exec line 508
exec line 308
exec line 460
exec line 461
exec line 462
exec line 463
ex

exec line 275
exec line 276
exec line 11
exec line 12
exec line 232
exec line 312
414 already visited
Try swapping line 351, acc=1449
exec line 352
exec line 353
exec line 354
exec line 517
exec line 518
exec line 519
exec line 520
exec line 521
exec line 528
exec line 529
exec line 530
exec line 531
exec line 600
exec line 601
exec line 602
exec line 537
exec line 538
exec line 43
exec line 44
exec line 45
exec line 46
exec line 331
exec line 332
exec line 333
exec line 159
exec line 160
exec line 161
exec line 162
exec line 163
exec line 76
exec line 77
exec line 78
exec line 79
exec line 80
exec line 126
exec line 127
exec line 508
exec line 308
exec line 460
exec line 461
exec line 462
exec line 463
exec line 464
exec line 543
exec line 544
exec line 545
exec line 70
exec line 71
exec line 363
exec line 364
exec line 60
exec line 61
exec line 62
exec line 63
exec line 64
exec line 510
exec line 511
exec line 512
exec line 513
exec line 514
exec line 21
exec line 22
exec line 474
ex

Try swapping line 607, acc=1384
exec line 130
exec line 131
exec line 132
exec line 557
exec line 558
exec line 559
exec line 560
exec line 384
exec line 385
exec line 174
exec line 175
exec line 176
exec line 177
exec line 84
exec line 85
exec line 600
exec line 601
exec line 602
exec line 537
exec line 538
exec line 43
exec line 44
exec line 45
exec line 46
exec line 331
exec line 332
exec line 333
exec line 159
exec line 160
exec line 161
exec line 162
exec line 163
exec line 76
exec line 77
exec line 78
exec line 79
exec line 80
exec line 126
exec line 127
exec line 508
exec line 308
exec line 460
exec line 461
exec line 462
exec line 463
exec line 464
exec line 543
exec line 544
exec line 545
exec line 70
exec line 71
exec line 363
exec line 364
exec line 60
exec line 61
exec line 62
exec line 63
exec line 64
exec line 510
exec line 511
exec line 512
exec line 513
exec line 514
exec line 21
exec line 22
exec line 474
exec line 475
exec line 476
exec line 496
exec line 429
exec lin

exec line 627
exec line 391
exec line 25
exec line 26
exec line 27
exec line 28
exec line 424
exec line 425
exec line 141
exec line 33
exec line 34
exec line 35
exec line 36
exec line 37
exec line 407
exec line 408
exec line 409
exec line 410
exec line 411
exec line 607
exec line 608
exec line 609
exec line 610
exec line 347
exec line 348
exec line 349
exec line 350
exec line 351
exec line 345
exec line 16
exec line 17
exec line 18
exec line 19
exec line 585
exec line 586
exec line 96
exec line 97
exec line 98
exec line 222
exec line 487
exec line 488
exec line 478
exec line 479
exec line 480
exec line 224
exec line 225
exec line 226
exec line 227
exec line 208
exec line 209
exec line 210
exec line 211
exec line 212
exec line 316
exec line 317
exec line 296
exec line 297
exec line 298
exec line 112
exec line 113
exec line 114
exec line 115
exec line 52
exec line 53
exec line 54
exec line 104
exec line 105
exec line 106
exec line 107
exec line 568
exec line 569
exec line 570
exec line 1

exec line 54
exec line 104
exec line 105
exec line 106
exec line 107
exec line 568
exec line 569
exec line 570
exec line 130
exec line 131
exec line 132
exec line 557
exec line 558
exec line 559
exec line 560
exec line 384
exec line 385
exec line 174
exec line 175
exec line 176
exec line 177
exec line 84
exec line 85
exec line 600
exec line 601
exec line 602
exec line 537
exec line 538
exec line 43
exec line 44
exec line 45
exec line 46
exec line 331
exec line 332
exec line 333
exec line 159
exec line 160
exec line 161
exec line 162
exec line 163
exec line 76
exec line 77
exec line 78
exec line 79
exec line 80
exec line 126
exec line 127
exec line 508
exec line 308
exec line 460
exec line 461
exec line 462
exec line 463
exec line 464
exec line 543
exec line 544
exec line 545
exec line 70
exec line 71
exec line 363
exec line 364
exec line 60
exec line 61
exec line 62
exec line 63
exec line 64
exec line 510
exec line 511
exec line 512
exec line 513
exec line 514
exec line 21
exec line 22

exec line 401
exec line 337
exec line 338
exec line 339
exec line 340
exec line 341
exec line 183
exec line 184
exec line 185
exec line 625
exec line 626
exec line 627
exec line 391
exec line 25
exec line 26
exec line 27
exec line 28
exec line 424
exec line 425
exec line 141
exec line 33
exec line 34
exec line 35
exec line 36
exec line 37
exec line 407
exec line 408
exec line 409
exec line 410
exec line 411
exec line 607
exec line 608
exec line 609
exec line 610
exec line 347
exec line 348
exec line 349
exec line 350
exec line 351
exec line 345
exec line 16
exec line 17
exec line 18
exec line 19
exec line 585
exec line 586
exec line 96
exec line 97
exec line 98
exec line 222
exec line 487
exec line 488
exec line 478
exec line 479
exec line 480
exec line 224
exec line 225
exec line 226
exec line 227
exec line 208
exec line 209
exec line 210
exec line 211
exec line 212
exec line 316
exec line 317
exec line 296
exec line 297
exec line 298
exec line 112
exec line 113
exec line 114
exec lin

exec line 601
exec line 602
exec line 537
exec line 538
exec line 43
exec line 44
exec line 45
exec line 46
exec line 331
exec line 332
exec line 333
exec line 159
exec line 160
exec line 161
exec line 162
exec line 163
exec line 76
exec line 77
exec line 78
exec line 79
exec line 80
exec line 126
exec line 127
exec line 508
exec line 308
exec line 460
exec line 461
exec line 462
exec line 463
exec line 464
exec line 543
exec line 544
exec line 545
exec line 70
exec line 71
exec line 363
exec line 364
exec line 60
exec line 61
exec line 62
exec line 63
exec line 64
exec line 510
exec line 511
exec line 512
exec line 513
exec line 514
exec line 21
exec line 22
exec line 474
exec line 475
exec line 476
exec line 496
exec line 429
exec line 430
exec line 431
exec line 376
exec line 377
exec line 378
exec line 379
exec line 282
exec line 283
exec line 284
exec line 285
exec line 286
exec line 483
exec line 484
exec line 485
exec line 562
exec line 563
exec line 564
exec line 565
exec line 

758